Using long sample data set for Olive Garden resturant chain

In [5]:
import pandas as pd
import numpy as np

focal_long = pd.read_csv("long_sample_csv.csv")
focal_long
print(focal_long.columns)

Index(['placekey', 'city', 'region', 'date_range_start', 'date_range_end',
       'raw_visit_counts', 'visits_by_day', 'safegraph_brand_ids',
       'naics_code', 'postal_code', 'brands', 'day', 'dailyvisits',
       'dayofweek', 'manyvisits', 'core_biz_area', 'weekend'],
      dtype='object')


Part 1: Define your benchmark comparison 

 

Question 1a (1 point): Choose another restaurant chain in the most original nationwide data as the benchmark for your own restaurant chain. This benchmark chain could be a major competitor of your own chain, a chain that your chain aspires to grow into, or a chain that your chain may want to acquire for future business. Whatever your choice is, explain why you make this choice and what you would like to learn most by comparing your own chain with this benchmark chain.

I selected Red Lobster as my benchmark chain. This is because similar to how Olive Garden specializes in a specific type of cuisine, Italian food, Red Lobster also specializes in a specific type of cuisine, seafood. By comparing my own chain with this benchmark chain, Olive Garden could learn more about having a strong brand presence, enhancing customer loyalty, and making more strategic decisions around menu creation, advertising and marketing, and brand promotion

Question 1b (1 point): once you choose your benchmark chain, repeat the data generating process of Project 1 for this benchmark chain. The resulting long sample -- let us call it "Benchmark Long" -- should have exactly the same data structure as the long sample you have created for your own chain at the end of Project 1. For ease of illustration, I will refer to the long sample of your own chain as "Focal Long". 

In [4]:
#import raw data
data_2018 = pd.read_csv("weekly_patterns_2018_sample.csv")
#data_2018
data_2019 = pd.read_csv("weekly_patterns_2019_sample.csv")
#data_2019
data_2020 = pd.read_csv("weekly_patterns_2020_sample.csv")
#data_2020
data_2021 = pd.read_csv("weekly_patterns_2021_sample.csv")
#data_2021
data_2022 = pd.read_csv("weekly_patterns_2022_sample.csv")
#data_2022

#subset
df_2018 = data_2018[data_2018['brands'] == 'Red Lobster']
df_2019 = data_2019[data_2019['brands'] == 'Red Lobster']
df_2020 = data_2020[data_2020['brands'] == 'Red Lobster']
df_2021 = data_2021[data_2021['brands'] == 'Red Lobster']
df_2022 = data_2022[data_2022['brands'] == 'Red Lobster']
wide_sample = pd.concat([df_2018, df_2019, df_2020, df_2021,df_2022])

#clean date_range_start variable
wide_sample['date_range_start'] = pd.to_datetime(wide_sample['date_range_start'],utc = True).dt.normalize()
wide_sample['date_range_end']=  pd.to_datetime(wide_sample['date_range_end'], utc = True).dt.normalize()

#split visits_by_day
visits_by_day_lists = wide_sample['visits_by_day'].str.replace('[\[\]]', '', regex=True).str.split(',', expand=True)
visits_by_day_lists

for i in range(7):
    wide_sample[f'daily_visits{i+1}'] = visits_by_day_lists[i].astype(int)

#reshape data/converting daily visits to integer
id_vars = ['placekey', 'city', 'region', 'date_range_start', 'date_range_end',
           'raw_visit_counts', 'visits_by_day', 'safegraph_brand_ids',
           'naics_code', 'postal_code', 'brands']

# Define the value_vars (these are the daily visits columns to be melted)
value_vars = ['daily_visits1', 'daily_visits2', 'daily_visits3', 'daily_visits4',
              'daily_visits5', 'daily_visits6', 'daily_visits7']

benchmark_long = wide_sample.melt(id_vars=id_vars, value_vars=value_vars, var_name='day', value_name='dailyvisits')
summary_stats = benchmark_long['dailyvisits'].describe()

#create dayofweek
benchmark_long['dayofweek'] = benchmark_long['day'].str[-1].astype(int)

days_of_week = {1: 'Monday', 2: 'Tuesday',
                3: 'Wednesday', 4: 'Thursday',
                5: 'Friday', 6: 'Saturday', 7: 'Sunday'}
benchmark_long['dayofweek'] = benchmark_long['dayofweek'].map(days_of_week)

#manyvisits
threshold = benchmark_long['dailyvisits'].mean()

benchmark_long['manyvisits'] = (benchmark_long['dailyvisits'] > threshold).astype(int)

#core_biz_area, and geographic variables
region_frequency = benchmark_long['region'].value_counts()
region_frequency

threshold = region_frequency.mean()

# Define the core business areas based on the threshold
# the regions above the mean is 1 and the regions under the mean is 0
benchmark_long['core_biz_area'] = benchmark_long['region'].apply(lambda x: 1 if region_frequency[x] >= threshold else 0)
benchmark_long
benchmark_long.columns
# Save the DataFrame to a CSV file without the index
benchmark_long.to_csv('benchmark_long.csv', index=False)


What is the unit of observation in each long sample? How many observations do you have in “Benchmark Long" and "Focal Long" respectively?

the unit of observation in focal long is placekey and the unit of observations in benchmark long is also placekey

number of observations in focal long and benchmark long

In [8]:
# Get the number of observations (rows) in the DataFrame
num_observations_benchmark = benchmark_long.shape[0]
print("Number of observations benchmark long -> ", num_observations_benchmark) #269612
num_observations_focal = focal_long.shape[0]
print("Number of observations focal long -> ", num_observations_focal) #354046

Number of observations benchmark long ->  269612
Number of observations focal long ->  354046


Question 1c (1 point): Do some simple comparison between the two long samples for sanity check. How many unique chain units do you have in “Benchmark Long" and "Focal Long" respectively? In how many unique states do your own chain and benchmark chain operate respectively? Are they consistent with your expectation?
 

In [9]:
unique_units_focal = focal_long['placekey'].nunique()

# Get unique states in "Focal Long"
unique_states_focal = focal_long['region'].nunique()

print("Unique chain units in Focal Long:", unique_units_focal) #201
print("Unique states in Focal Long:", unique_states_focal) #43

unique_units_benchmark = benchmark_long['placekey'].nunique()
unique_states_benchmark = benchmark_long['region'].nunique()

print("Unique chain units in Benchmark Long:", unique_units_benchmark) #149
print("Unique states in Benchmark Long:", unique_states_benchmark) #38

#Results are consistent with expectation

Unique chain units in Focal Long: 201
Unique states in Focal Long: 43
Unique chain units in Benchmark Long: 149
Unique states in Benchmark Long: 38


Part 2: Merge data for comparison

Question 2a (2 points): To compare the two chains in each state-day, collapse each long sample into a new dataset by "state-day". We will call these two new datasets "BenchmarkSummary" and "FocalSummary". Each of them describes that, for the relevant chain, the number of unique chain units there are in a state-day (name the variable Nunits), the total dailyvisits are observed in that state-day (name the variable SumDailyVisits), and the maximum, minimum, and median dailyvisits observed across all store units within that state-day (name them as MaxDailyVisitsPerUnit, MinDailyVisitsPerUnit, and MedDailyVisitsPerUnit). 

In [10]:
# Group focal_long by state-day so region and day
#BenchmarkSummary = benchmark_long.groupby(['region', 'day']).agg(
#    Nunits=('placekey', 'nunique'),
#    SumDailyVisits=('dailyvisits', 'sum'),
#    MaxDailyVisitsPerUnit=('dailyvisits', 'max'),
#    MinDailyVisitsPerUnit=('dailyvisits', 'min'),
#    MedDailyVisitsPerUnit=('dailyvisits', 'median')
#).reset_index()

# Group focal_long by state-day so region and day
#FocalSummary = focal_long.groupby(['region', 'day']).agg(
#    Nunits=('placekey', 'nunique'),
#    SumDailyVisits=('dailyvisits', 'sum'),
#    MaxDailyVisitsPerUnit=('dailyvisits', 'max'),
#    MinDailyVisitsPerUnit=('dailyvisits', 'min'),
#    MedDailyVisitsPerUnit=('dailyvisits', 'median')
#).reset_index()

BenchmarkSummary = benchmark_long.groupby(['region', 'date_range_start']).agg(
    Nunits=('placekey', 'nunique'),
    SumDailyVisits=('dailyvisits', 'sum'),
    MaxDailyVisitsPerUnit=('dailyvisits', 'max'),
    MinDailyVisitsPerUnit=('dailyvisits', 'min'),
    MedDailyVisitsPerUnit=('dailyvisits', 'median')
).reset_index()

FocalSummary = focal_long.groupby(['region', 'date_range_start']).agg(
    Nunits=('placekey', 'nunique'),
    SumDailyVisits=('dailyvisits', 'sum'),
    MaxDailyVisitsPerUnit=('dailyvisits', 'max'),
    MinDailyVisitsPerUnit=('dailyvisits', 'min'),
    MedDailyVisitsPerUnit=('dailyvisits', 'median')
).reset_index()



What is the unit of observation in these two summary datasets? How many observations are there in BenchmarkSummary? How many observations are there in FocalSummary? Why do you think the number of observations are similar (or different)? Are they consistent with your expectation? 

unit of observation in two summary datasets, is a state-day pair.

In [11]:
print("Printing Focal Summary")
FocalSummary

Printing Focal Summary


,region,date_range_start,Nunits,SumDailyVisits,MaxDailyVisitsPerUnit,MinDailyVisitsPerUnit,MedDailyVisitsPerUnit
0,AL,2018-01-01 00:00:00+00:00,2,428,51,7,29.0
1,AL,2018-01-08 00:00:00+00:00,2,399,51,11,28.0
2,AL,2018-01-15 00:00:00+00:00,2,410,66,4,28.5
3,AL,2018-01-22 00:00:00+00:00,2,497,70,17,31.0
4,AL,2018-01-29 00:00:00+00:00,2,464,58,16,31.0
...,...,...,...,...,...,...,...
10826,WV,2022-11-28 00:00:00+00:00,1,356,106,24,41.0
10827,WV,2022-12-05 00:00:00+00:00,1,386,120,28,38.0
10828,WV,2022-12-12 00:00:00+00:00,1,404,92,37,45.0
10829,WV,2022-12-19 00:00:00+00:00,1,343,87,7,58.0


In [12]:
print("Printing Benchmark Summary")
BenchmarkSummary

Printing Benchmark Summary


,region,date_range_start,Nunits,SumDailyVisits,MaxDailyVisitsPerUnit,MinDailyVisitsPerUnit,MedDailyVisitsPerUnit
0,AL,2018-01-01 00:00:00+00:00,4,316,39,1,8.5
1,AL,2018-01-08 00:00:00+00:00,4,344,35,1,10.0
2,AL,2018-01-15 00:00:00+00:00,4,398,48,0,13.0
3,AL,2018-01-22 00:00:00+00:00,4,356,34,1,11.0
4,AL,2018-01-29 00:00:00+00:00,4,384,49,1,11.0
...,...,...,...,...,...,...,...
9878,WI,2022-11-28 00:00:00+00:00,3,242,31,3,11.0
9879,WI,2022-12-05 00:00:00+00:00,3,242,31,1,10.0
9880,WI,2022-12-12 00:00:00+00:00,3,254,29,5,10.0
9881,WI,2022-12-19 00:00:00+00:00,3,192,20,1,9.0


In [13]:
num_observations_benchmark = len(BenchmarkSummary)
num_observations_focal = len(FocalSummary)

print("Number of observations in BenchmarkSummary:", num_observations_benchmark)
print("Number of observations in FocalSummary:", num_observations_focal)


Number of observations in BenchmarkSummary: 9883
Number of observations in FocalSummary: 10831


the number of observations are slightly different when they are collasped into a new data-set by state-day. this makes sense because when defining the intial datasets the number of observations in the focal_long dataset > number of observations in the benchmark_long dataset. In addition, the number of unique chain units and the number of unique states in focal_long is > the number of unique chain units and the number of unique states in benchmark_long. so it is consistent with my expectation

Question 2b (2 points): Merge BenchmarkSummary and FocalSummary by state-day. How many observations can be matched between the two datasets? How many observations appear in BenchmarkSummary but cannot be matched with FocalSummary? How many observations appear in FocalSummary but cannot be matched with BenchmarkSummary? Why do you observe imperfect matches? Provide explanations for both the matched parts and the unmatched parts. 

From now on, only keep the observations that match between BenchmarkSummary and FocalSummary. Let us call this new data MergedSummary. 

In [14]:
BenchmarkSummary['date_range_start'] = pd.to_datetime(BenchmarkSummary['date_range_start'])
FocalSummary['date_range_start'] = pd.to_datetime(FocalSummary['date_range_start'])

MergedSummary = pd.merge(BenchmarkSummary, FocalSummary, on=['region', 'date_range_start'], suffixes=('_Benchmark', '_Focal'), how='inner')

# Number of matched observations
num_matched_observations = len(MergedSummary)

# Number of observations in BenchmarkSummary that can't be matched with FocalSummary
num_unmatched_benchmark = len(BenchmarkSummary) - num_matched_observations

# Number of observations in FocalSummary that can't be matched with BenchmarkSummary
num_unmatched_focal = len(FocalSummary) - num_matched_observations

print("Number of obs matched between BenchmarkSummary and FocalSummary:", num_matched_observations)
print("Number of obs in BenchmarkSummary but can't be matched with FocalSummary:", num_unmatched_benchmark)
print("Number of obs in FocalSummary but can't be matched with BenchmarkSummary:", num_unmatched_focal)

MergedSummary

Number of obs matched between BenchmarkSummary and FocalSummary: 8999
Number of obs in BenchmarkSummary but can't be matched with FocalSummary: 884
Number of obs in FocalSummary but can't be matched with BenchmarkSummary: 1832


,region,date_range_start,Nunits_Benchmark,SumDailyVisits_Benchmark,MaxDailyVisitsPerUnit_Benchmark,MinDailyVisitsPerUnit_Benchmark,MedDailyVisitsPerUnit_Benchmark,Nunits_Focal,SumDailyVisits_Focal,MaxDailyVisitsPerUnit_Focal,MinDailyVisitsPerUnit_Focal,MedDailyVisitsPerUnit_Focal
0,AL,2018-01-01 00:00:00+00:00,4,316,39,1,8.5,2,428,51,7,29.0
1,AL,2018-01-08 00:00:00+00:00,4,344,35,1,10.0,2,399,51,11,28.0
2,AL,2018-01-15 00:00:00+00:00,4,398,48,0,13.0,2,410,66,4,28.5
3,AL,2018-01-22 00:00:00+00:00,4,356,34,1,11.0,2,497,70,17,31.0
4,AL,2018-01-29 00:00:00+00:00,4,384,49,1,11.0,2,464,58,16,31.0
...,...,...,...,...,...,...,...,...,...,...,...,...
8994,WI,2022-11-28 00:00:00+00:00,3,242,31,3,11.0,1,199,49,14,23.0
8995,WI,2022-12-05 00:00:00+00:00,3,242,31,1,10.0,1,227,48,18,28.0
8996,WI,2022-12-12 00:00:00+00:00,3,254,29,5,10.0,1,180,36,16,27.0
8997,WI,2022-12-19 00:00:00+00:00,3,192,20,1,9.0,1,169,36,0,27.0


Imperfect matches happen because there can be variations within the two datasets and this can lead to differences in the data collection process. In the matched parts, the matched observations count for the state-day combinations where both Benchmark and Focal Summary have data available, which allows for direct comparison between the two sets of data. In the unmatched parts, observations that appear in Benchmark Summary and not in Focal Summary and the other way around observations that appear in Focal Summary and not in Benchmark Sumary are due to differences in the available data. this could be a result of missing data or variations in individual data collection

Part 3: Group comparison in the merged data

Question 3a (2 point): Now using MergedSummary, you can compare the two chains. Calculate average SumDailyVisits for the two chains.  Show the results in a table.  How does the average SumDailyVisits differ between the two chains? (Write at least one sentence about your takeaway from the table.)
 

In [15]:
average_sum_daily_visits_benchmark = MergedSummary['SumDailyVisits_Benchmark'].mean()
average_sum_daily_visits_focal = MergedSummary['SumDailyVisits_Focal'].mean()

visits_comparison_table = pd.DataFrame({
    'Chain': ['Benchmark', 'Focal'],
    'Average SumDailyVisits': [average_sum_daily_visits_benchmark, average_sum_daily_visits_focal]
})

visits_comparison_table


,Chain,Average SumDailyVisits
0,Benchmark,448.061007
1,Focal,1206.787754


One sentence takeaway: the average SumDailyVisits for Focal chain is greater than the average SumDailyVisits for Benchmark chain. More people visit Olive Garden on the Daily on average then Red Lobster.

Question 3b (2 points): Perform a statistical test on the difference of average SumDailyVisits between the two chains. Is the difference statistically significant from zero with 95% confidence? Does it confirm your prior in Question 3a?

In [16]:
from scipy.stats import ttest_ind

sum_daily_visits_benchmark = MergedSummary['SumDailyVisits_Benchmark']
sum_daily_visits_focal = MergedSummary['SumDailyVisits_Focal']

t_statistic, p_value = ttest_ind(sum_daily_visits_benchmark, sum_daily_visits_focal)

alpha = 0.05
if p_value < alpha:
    print("The difference in average SumDailyVisits between the two chains is statistically significant")
    print("Since it is statistically signficant, confirms 3a) avg SumDailyVisits Focal > Benchmark")
else:
    print("The difference in average SumDailyVisits between the two chains is not statistically significant")

MergedSummary

The difference in average SumDailyVisits between the two chains is statistically significant
Since it is statistically signficant, confirms 3a) avg SumDailyVisits Focal > Benchmark


,region,date_range_start,Nunits_Benchmark,SumDailyVisits_Benchmark,MaxDailyVisitsPerUnit_Benchmark,MinDailyVisitsPerUnit_Benchmark,MedDailyVisitsPerUnit_Benchmark,Nunits_Focal,SumDailyVisits_Focal,MaxDailyVisitsPerUnit_Focal,MinDailyVisitsPerUnit_Focal,MedDailyVisitsPerUnit_Focal
0,AL,2018-01-01 00:00:00+00:00,4,316,39,1,8.5,2,428,51,7,29.0
1,AL,2018-01-08 00:00:00+00:00,4,344,35,1,10.0,2,399,51,11,28.0
2,AL,2018-01-15 00:00:00+00:00,4,398,48,0,13.0,2,410,66,4,28.5
3,AL,2018-01-22 00:00:00+00:00,4,356,34,1,11.0,2,497,70,17,31.0
4,AL,2018-01-29 00:00:00+00:00,4,384,49,1,11.0,2,464,58,16,31.0
...,...,...,...,...,...,...,...,...,...,...,...,...
8994,WI,2022-11-28 00:00:00+00:00,3,242,31,3,11.0,1,199,49,14,23.0
8995,WI,2022-12-05 00:00:00+00:00,3,242,31,1,10.0,1,227,48,18,28.0
8996,WI,2022-12-12 00:00:00+00:00,3,254,29,5,10.0,1,180,36,16,27.0
8997,WI,2022-12-19 00:00:00+00:00,3,192,20,1,9.0,1,169,36,0,27.0


Part 4: Basic regression analysis 

Question 4a (1 points): Focus on the data MergedSummary. Define a dummy “Post03132020” equal to one if date is on or after March 13, 2020. Define a dummy "BlueState" equal to 1 if a state has a higher fraction of population voting for Hilary Clinton according to the 2016 Presidential Election outcome (see NY Times Links to an external site.for the color of each state). Define “Blue_Post” as BlueState times Post03132020. Regress ln(SumDailyVisits+1) of your own chain on BlueState, Post03132020, and Blue_Post.

In [17]:
import statsmodels.api as sm

MergedSummary['Post03132020'] = (MergedSummary['date_range_start'] >= ('2020-03-13')).astype(int)
MergedSummary

Red_Blue_States = pd.read_csv("RedBlueStates.csv")
Red_Blue_States

Red_Blue_States['BlueState'] = (Red_Blue_States['Color'] == 'Blue').astype(int)

state_color_mapping = Red_Blue_States.set_index('State')['BlueState'].to_dict()

MergedSummary['BlueState'] = MergedSummary['region'].map(state_color_mapping)


MergedSummary['Blue_Post'] = MergedSummary['BlueState'] * MergedSummary['Post03132020']


MergedSummary['ln_SumDailyVisits'] = np.log(MergedSummary['SumDailyVisits_Focal'] + 1)


X = MergedSummary[['BlueState', 'Post03132020', 'Blue_Post']]
X = sm.add_constant(X)  
y = MergedSummary['ln_SumDailyVisits']


model = sm.OLS(y, X).fit()

model.summary()
MergedSummary.to_csv('MergedSummary.csv', index=False)



Question 4a (1 points): Focus on the data MergedSummary. Define a dummy “Post03132020” equal to one if date is on or after March 13, 2020. Define a dummy "BlueState" equal to 1 if a state has a higher fraction of population voting for Hilary Clinton according to the 2016 Presidential Election outcome (see NY Times Links to an external site.for the color of each state). Define “Blue_Post” as BlueState times Post03132020. Regress ln(SumDailyVisits+1) of your own chain on BlueState, Post03132020, and Blue_Post. 

Comment on the estimated coefficient for each of the three variables: Are they statistically significant from zero with 95% confidence? What is the economic meaning of each coefficient? Do you accept or reject the null hypothesis “H0: The national emergency announced on March 13, 2020 does not make a difference on average SumDailyVisits of your own chain between red and blue states”?

coefficent for BlueState is -0.1589, coefficient for Post03132020 is 0.0357, and the coefficent for Blue_Post -0.4065. Economic Meaning for blue state: coefficient is statistically significant meaning that being in a blue state have fewer visits on average compared to red states, holding all other factors constant. Economic meaning for Post03132020 is 0.0357 and is not statistically signficant -> economically meaning that there is not enough evidence to suggest that the number of daily visits has changed after march 13 2020. Blue Post coefficient is also negative and less than p <0.05 meaning that the combined effect of being a blue state and the period after march 13th 2020 is associated with further decrease in natural logarithm of daily visits plus one meaning that on average number of daily visits is lower in the Blue Post case. In terms of the null hypothesis: H0: The national emergency announced on March 13, 2020, does not make a difference on average SumDailyVisits of your own chain between red and blue states" , we can reject it because the Blue Post coefficent is signficant and has a p value < 0.05. the non signficant POST03132020 variable shows that a national emergency solemly didn't change the average SumDailyVisits.

Question 4b (1 points): How would your answer to Question 4a change if (1) you add state fixed effects? (2) you add date fixed effects? and (3) you add both state fixed effects and date fixed effects? Which specification would you prefer for the hypothesis testing?

If I add state fixed effects, I would have dummy variables for each state, and this would help me look at time-invariant differeces that can influence the dependent variable, average number of total visits. Likewise, addition of date fixed effects involve using dummy variables for each date, which addresses time-specific factors that also affect the average SumDailyVisits. By controlling for other time specific facts this will show that the coefficent estimates for BlueState,Post03132020, and Blue_Post will reflect the average effect across dates after adjusting for the date-specific factors through adding date fixed effects. It would depend on the hypothesis being tested, state fixed effects benefit understanding effect of avriables while controlling for state-specific factors, while date fixed effects are valuable for addressing time specific factors. the one to be prefered depends on the hypothesis being tested.

Question 4c (2 points): Now suppose you replace the dependent variable as [ln(SumDailyVisits+1) of your own chain - ln(SumDailyVisits+1) of your benchmark chain]. How would your findings in Question 4a and Question 4b change? What may explain these changes?

It would change the interpretation of the coefficents and the conclusions drawn from 4a and 4b. In 4a, the coefficents represented the effects of BlueState, Post03132020, and their interaction term (Blue_Post) on the logarithm of visits for your chain, holding other factors constant. In 4c howevever, the coefficents represent the effects of BlueState, Post03132020, and the interaction term (Blue_Post) on the difference in visits between my focal and benchmark chain as the dependent variable is the difference between the natural logarithm of SumDailyVisits plus one of focal and benchmark chain. in 4c, the statsitcical test would examine whether or not there is a statistically signficant difference in the differences of SumDailyVisits in focal and benchmark chain in comparison to the difference in average SumDailyVisits between chain and benchmark chain. These changes that can occur can be explained by the change in the dependent variable. 4c focuses more on the relative performance between the two chains.